# Comprehensive Stock Portfolio Analysis

This notebook provides an extensive analysis of a stock portfolio, including performance, risk, asset allocation, correlation, and benchmark comparison. The analysis is visualized to make the information easily understandable.


## 1. Data Collection

In this section, we collect historical stock price data using `yfinance`.


In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Define the portfolio
portfolio = {
    'NVDA': {'shares': 10, 'avg_price': 150},
    'MSFT': {'shares': 15, 'avg_price': 200},
    'GOOGL': {'shares': 8, 'avg_price': 1200}
}

# Fetch historical data
symbols = list(portfolio.keys())
start_date = "2020-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')
data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']

# Display the first few rows of the data
data.head()


## 2. Portfolio Performance

### Cumulative Returns vs. Benchmark

We compare the cumulative returns of the portfolio and individual stocks against a market benchmark (S&P 500).


In [2]:
# Define benchmark
benchmark_symbol = '^GSPC'  # S&P 500

# Fetch benchmark data
benchmark_data = yf.download(benchmark_symbol, start=start_date, end=end_date)['Adj Close']

# Calculate cumulative returns
returns = data.pct_change().dropna()
portfolio_returns = (returns + 1).cumprod() - 1
benchmark_returns = (benchmark_data.pct_change().dropna() + 1).cumprod() - 1
portfolio_cumulative_returns = portfolio_returns.mean(axis=1)
# Plot cumulative returns
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
for sym in symbols:
    plt.plot(portfolio_returns[sym], label=sym)

plt.plot(portfolio_cumulative_returns, label='Cumulative', linewidth=3 )
plt.plot(benchmark_returns, label='S&P 500', linewidth=3, linestyle='--')
plt.title('Cumulative Returns vs. Benchmark')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()


## 3. Risk Analysis

### Daily Returns Distribution

We analyze the distribution of daily returns for the portfolio.


In [3]:
# Calculate daily returns for the portfolio
weights = [portfolio[sym]['shares'] * portfolio[sym]['avg_price'] for sym in symbols]
total_value = sum(weights)
weights = [w / total_value for w in weights]
portfolio_returns = returns.dot(weights)

# Plot daily returns distribution
import seaborn as sns

plt.figure(figsize=(14, 7))
sns.histplot(portfolio_returns, bins=50, kde=True)
plt.title('Portfolio Daily Returns Distribution')
plt.xlabel('Daily Returns')
plt.ylabel('Frequency')
plt.show()


### Risk-Return Scatter Plot

We plot the risk (standard deviation) versus return for the portfolio and individual stocks.


In [4]:
# Calculate annualized return and volatility
annualized_return = returns.mean() * 252
annualized_volatility = returns.std() * (252**0.5)

# Data for scatter plot
risk_return_data = pd.DataFrame({
    'Return': annualized_return,
    'Volatility': annualized_volatility
})

# Scatter plot
plt.figure(figsize=(14, 7))
sns.scatterplot(x='Volatility', y='Return', data=risk_return_data, s=100)
plt.title('Risk-Return Scatter Plot')
plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Return')
plt.show()


## 4. Asset Allocation

### Current Investment Allocation

We show the proportion of investment in each stock within the portfolio.


In [5]:
# Define portfolio
portfolio_values = {sym: portfolio[sym]['shares'] * portfolio[sym]['avg_price'] for sym in symbols}

# Pie chart of current investment
labels = portfolio_values.keys()
sizes = portfolio_values.values()

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title('Current Investment Allocation')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()


## 5. Correlation Analysis

We analyze the correlation between the returns of different stocks in the portfolio.


In [6]:
# Correlation matrix
correlation_matrix = returns.corr()

# Plot correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Stock Correlation Matrix')
plt.show()


## 6. Advanced Risk Metrics

### Conditional Value at Risk (CVaR)

Assess the expected loss exceeding VaR, providing insights into tail risk.


In [7]:
import numpy as np

# Calculate Value at Risk (VaR)
confidence_level = 0.95
VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))

# Calculate Conditional Value at Risk (CVaR)
CVaR = portfolio_returns[portfolio_returns <= VaR].mean()

# Plot VaR and CVaR
plt.figure(figsize=(14, 7))
sns.histplot(portfolio_returns, bins=50, kde=True, color='blue')
plt.axvline(VaR, color='red', linestyle='--', label=f'VaR ({confidence_level*100}%): {VaR:.2f}')
plt.axvline(CVaR, color='orange', linestyle='--', label=f'CVaR: {CVaR:.2f}')
plt.title('Portfolio Daily Returns Distribution with VaR and CVaR')
plt.xlabel('Daily Returns')
plt.ylabel('Frequency')
plt.legend()
plt.show()


### Rolling Beta

Analyze the portfolio's beta changes over time to assess changing sensitivity to the market.


In [8]:
window_size = 252  # 1 year of trading days

# Calculate rolling beta
rolling_beta = returns.rolling(window=window_size).cov(benchmark_returns) / benchmark_returns.rolling(window=window_size).var()

# Plot rolling beta
plt.figure(figsize=(14, 7))
for sym in symbols:
    plt.plot(rolling_beta[sym], label=sym)
plt.title('Rolling Beta (1 Year Window)')
plt.xlabel('Date')
plt.ylabel('Beta')
plt.legend()
plt.show()


## 7. Drawdown Analysis

### Maximum Drawdown

Measure the largest peak-to-trough decline to understand the risk of significant losses.


In [9]:
# Calculate drawdowns
cumulative_returns = (portfolio_returns + 1).cumprod()
rolling_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - rolling_max) / rolling_max

# Calculate maximum drawdown
max_drawdown = drawdown.min()

# Plot drawdowns
plt.figure(figsize=(14, 7))
plt.plot(drawdown, label='Drawdown')
plt.axhline(max_drawdown, color='red', linestyle='--', label=f'Max Drawdown: {max_drawdown:.2f}')
plt.title('Portfolio Drawdown')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.legend()
plt.show()
